In [1]:
import pandas as pd
pd.set_option('display.max_rows', 15)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import operator

In [2]:
data = pd.read_csv('./nba_data.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data_train = pd.DataFrame(data[data.Year < 2017])
data_test = pd.DataFrame(data[data.Year == 2017])
data['Champion'] = data['Champion'] * 1
data.set_value(1312, 'Champion', 1)

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,2P%,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.417,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics
1,Golden State Warriors,1960.0,49.0,0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.409,0.669,78.9,23.9,0.0,0.0,0.0,118.6,9.0,Warriors
2,Philadelphia 76ers,1960.0,45.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.414,0.791,72.1,22.3,0.0,0.0,0.0,118.9,5.0,76ers
3,New York Knicks,1960.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.421,0.765,70.0,22.2,0.0,0.0,0.0,117.3,5.0,Knicks
4,Atlanta Hawks,1960.0,46.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.419,0.745,71.2,25.1,0.0,0.0,0.0,113.4,10.0,Hawks
5,Detroit Pistons,1960.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.397,0.729,73.2,19.6,0.0,0.0,0.0,111.6,3.0,Pistons
6,Los Angeles Lakers,1960.0,25.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.386,0.730,72.4,19.3,0.0,0.0,0.0,107.3,7.0,Lakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,Denver Nuggets,2017.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.519,0.774,46.4,25.3,6.9,4.0,14.9,111.7,1.0,Nuggets
1321,New Orleans Pelicans,2017.0,34.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.495,0.750,43.7,22.8,7.8,5.5,12.9,104.3,5.0,Pelicans


In [3]:
data

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,2P%,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.417,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics
1,Golden State Warriors,1960.0,49.0,0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.409,0.669,78.9,23.9,0.0,0.0,0.0,118.6,9.0,Warriors
2,Philadelphia 76ers,1960.0,45.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.414,0.791,72.1,22.3,0.0,0.0,0.0,118.9,5.0,76ers
3,New York Knicks,1960.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.421,0.765,70.0,22.2,0.0,0.0,0.0,117.3,5.0,Knicks
4,Atlanta Hawks,1960.0,46.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.419,0.745,71.2,25.1,0.0,0.0,0.0,113.4,10.0,Hawks
5,Detroit Pistons,1960.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.397,0.729,73.2,19.6,0.0,0.0,0.0,111.6,3.0,Pistons
6,Los Angeles Lakers,1960.0,25.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.386,0.730,72.4,19.3,0.0,0.0,0.0,107.3,7.0,Lakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,Denver Nuggets,2017.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.519,0.774,46.4,25.3,6.9,4.0,14.9,111.7,1.0,Nuggets
1321,New Orleans Pelicans,2017.0,34.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.495,0.750,43.7,22.8,7.8,5.5,12.9,104.3,5.0,Pelicans


In [4]:
inputs = ['Wins', 'MVP', 'Scoring Leader', 'Rebound Leader', 
          'Assist Leader', 'WS Leader', 'DPOY', 'MIP', '6MOY', 
          'Coach of Year', 'All-Stars', 'All-Defensive', 'All-NBA', 
          'FG%', '3P Attempts', '3P%', '2P Attempts', '2P%', 'FT%',
          'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades', ]

for inp in inputs:
    data_train[inp] = preprocessing.scale(data_train[inp])
    data_test[inp] = preprocessing.scale(data_test[inp])


In [5]:
model = LogisticRegression()
inputs = ['Wins', 'MVP', 'Scoring Leader', 'Rebound Leader', 
          'Assist Leader', 'WS Leader', 'DPOY', 'MIP', '6MOY', 
          'Coach of Year', 'All-Stars', 'All-Defensive', 'All-NBA', 
          'FG%', '3P Attempts', '3P%', '2P Attempts', '2P%', 'FT%',
          'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades', ]

model.fit(data_train[inputs], data_train['Champion'])
model.coef_

array([[ 1.86702629,  0.15394087,  0.01770719, -0.06450039, -0.17370371,
        -0.03809625,  0.20552379, -0.16252231,  0.06067736,  0.04415351,
        -0.13284776,  0.18061622,  0.3623431 , -0.43780833, -0.31507895,
        -0.48742802,  0.03638233,  0.64712273, -0.15285073,  0.3661041 ,
        -0.05607334, -0.30090762, -0.04571321,  0.39339692, -0.34690444,
         0.21606577]])

It looks like having the scoring leader, the sixth man of the year, and coach of the year greatly increase your chance of winning the finals. 

In [6]:
data_train.groupby('Year')[inputs]

In [7]:
model = RandomForestClassifier()

model.fit(data[inputs], data['Champion'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [8]:
model.predict_proba(data[inputs])

array([[ 0. ,  1. ],
       [ 0.8,  0.2],
       [ 1. ,  0. ],
       ..., 
       [ 1. ,  0. ],
       [ 1. ,  0. ],
       [ 1. ,  0. ]])

In [9]:
data

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,2P%,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.417,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics
1,Golden State Warriors,1960.0,49.0,0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.409,0.669,78.9,23.9,0.0,0.0,0.0,118.6,9.0,Warriors
2,Philadelphia 76ers,1960.0,45.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.414,0.791,72.1,22.3,0.0,0.0,0.0,118.9,5.0,76ers
3,New York Knicks,1960.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.421,0.765,70.0,22.2,0.0,0.0,0.0,117.3,5.0,Knicks
4,Atlanta Hawks,1960.0,46.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.419,0.745,71.2,25.1,0.0,0.0,0.0,113.4,10.0,Hawks
5,Detroit Pistons,1960.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.397,0.729,73.2,19.6,0.0,0.0,0.0,111.6,3.0,Pistons
6,Los Angeles Lakers,1960.0,25.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.386,0.730,72.4,19.3,0.0,0.0,0.0,107.3,7.0,Lakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,Denver Nuggets,2017.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.519,0.774,46.4,25.3,6.9,4.0,14.9,111.7,1.0,Nuggets
1321,New Orleans Pelicans,2017.0,34.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.495,0.750,43.7,22.8,7.8,5.5,12.9,104.3,5.0,Pelicans


In [10]:
data[data.Team == 'Cleveland Cavaliers']

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,2P%,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID
118,Cleveland Cavaliers,1971.0,15.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.424,0.746,48.6,25.2,0.0,0.0,0.0,102.1,1.0,Cavaliers
135,Cleveland Cavaliers,1972.0,23.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.428,0.736,50.0,25.1,0.0,0.0,0.0,105.8,2.0,Cavaliers
152,Cleveland Cavaliers,1973.0,32.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.435,0.747,49.5,25.7,0.0,0.0,0.0,102.7,2.0,Cavaliers
169,Cleveland Cavaliers,1974.0,29.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.439,0.772,45.9,25.0,7.3,3.6,18.8,100.3,3.0,Cavaliers
185,Cleveland Cavaliers,1975.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.462,0.742,43.4,23.2,7.3,4.2,17.8,99.0,3.0,Cavaliers
201,Cleveland Cavaliers,1976.0,49.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.454,0.737,46.1,22.5,7.8,4.8,16.2,101.7,3.0,Cavaliers
223,Cleveland Cavaliers,1977.0,43.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.449,0.737,47.3,22.5,7.1,5.8,16.5,102.1,3.0,Cavaliers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,Cleveland Cavaliers,2011.0,19.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.461,0.745,40.3,21.0,6.6,4.2,14.2,95.5,1.0,Cavaliers
1154,Cleveland Cavaliers,2012.0,21.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.446,0.716,42.3,19.8,7.1,4.0,15.4,93.0,1.0,Cavaliers


In [11]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
asd = ['Accolades','PPG','RPG','APG']
model.fit(data[asd], data['Wins'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [12]:
data['pred_wins'] = model.predict(data[asd])

In [13]:
data

,Team,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,...,FT%,RPG,APG,STL,BLK,TOVPG,PPG,Accolades,Franchise ID,pred_wins
0,Boston Celtics,1960.0,59.0,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.734,80.2,24.7,0.0,0.0,0.0,124.5,13.0,Celtics,59.0
1,Golden State Warriors,1960.0,49.0,0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.669,78.9,23.9,0.0,0.0,0.0,118.6,9.0,Warriors,50.3
2,Philadelphia 76ers,1960.0,45.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.791,72.1,22.3,0.0,0.0,0.0,118.9,5.0,76ers,40.4
3,New York Knicks,1960.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.765,70.0,22.2,0.0,0.0,0.0,117.3,5.0,Knicks,34.0
4,Atlanta Hawks,1960.0,46.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.745,71.2,25.1,0.0,0.0,0.0,113.4,10.0,Hawks,47.6
5,Detroit Pistons,1960.0,30.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.729,73.2,19.6,0.0,0.0,0.0,111.6,3.0,Pistons,28.9
6,Los Angeles Lakers,1960.0,25.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.730,72.4,19.3,0.0,0.0,0.0,107.3,7.0,Lakers,27.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,Denver Nuggets,2017.0,40.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.774,46.4,25.3,6.9,4.0,14.9,111.7,1.0,Nuggets,35.4
1321,New Orleans Pelicans,2017.0,34.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.750,43.7,22.8,7.8,5.5,12.9,104.3,5.0,Pelicans,41.5


In [14]:
print(-cross_val_score(model, data[inputs], data['Wins'], cv= 10,
                       scoring='neg_mean_squared_error').mean())

0.0405799156983


In [15]:
(data['pred_wins'] - data['Wins']).mean()

-0.009344385832705275

In [16]:
(data['pred_wins'] - data['Wins']).std()

4.058574162249519

In [11]:
elo_data = pd.read_csv('./nbaallelo.csv')
elo_data = elo_data[elo_data.year_id > 1958]

In [12]:
elo_data

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
8460,4231,195810190SYR,NBA,1,1959,10/19/1958,1,0,DET,Pistons,...,37.834084,SYR,Sixers,103,1553.9590,1558.7543,A,L,0.278847,NaN
8461,4231,195810190SYR,NBA,0,1959,10/19/1958,1,0,SYR,Sixers,...,45.638824,DET,Pistons,94,1488.8943,1484.0990,H,W,0.721153,NaN
8462,4232,195810220LAL,NBA,0,1959,10/22/1958,1,0,MNL,Lakers,...,30.401035,CIN,Kings,79,1481.4917,1465.9398,H,W,0.520005,NaN
8463,4232,195810220LAL,NBA,1,1959,10/22/1958,1,0,CIN,Kings,...,35.948742,MNL,Lakers,99,1395.4003,1410.9521,A,L,0.479995,NaN
8464,4233,195810230STL,NBA,1,1959,10/23/1958,2,0,DET,Pistons,...,37.622662,STL,Hawks,104,1545.3391,1547.3676,A,L,0.283296,NaN
8465,4233,195810230STL,NBA,0,1959,10/23/1958,1,0,STL,Hawks,...,44.456135,DET,Pistons,103,1484.0990,1482.0706,H,W,0.716704,NaN
8466,4234,195810250CIN,NBA,0,1959,10/25/1958,2,0,CIN,Kings,...,36.701500,MNL,Lakers,94,1410.9521,1403.6812,H,W,0.709340,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126307,63154,201506090CLE,NBA,0,2015,6/9/2015,99,1,CLE,Cavaliers,...,61.826408,GSW,Warriors,91,1797.5032,1790.9591,H,W,0.527843,NaN
126308,63155,201506110CLE,NBA,1,2015,6/11/2015,101,1,GSW,Warriors,...,67.794983,CLE,Cavaliers,82,1723.4149,1704.3949,A,W,0.453428,NaN


In [19]:
print(elo_data[(elo_data.seasongame < 82) & (elo_data.fran_id == 'Celtics')].groupby('year_id')['pts'].mean())
elo_data[(elo_data.fran_id == 'Celtics')].groupby('year_id')['pts'].mean()

year_id
1959    117.209877
1960    123.654321
1961    119.580247
1962    121.037037
1963    118.962963
1964    112.901235
1965    112.740741
           ...    
2009    100.740741
2010     99.271605
2011     96.308642
2012     91.320988
2013     96.518519
2014     96.172840
2015    101.320988
Name: pts, dtype: float64


year_id
1959    117.289157
1960    122.818182
1961    119.853933
1962    120.170213
1963    118.698925
1964    111.955556
1965    113.315217
           ...    
2009    101.093750
2010     98.009434
2011     96.318681
2012     91.302326
2013     95.540230
2014     96.243902
2015    101.058140
Name: pts, dtype: float64

In [ ]:
teams = list(elo_data['fran_id'].unique())

In [ ]:
diffs = {}
for i in range(1,82):
    diffs[i] = []
for team in teams:
    for i in range(1,82):
        diffs[i].append(((elo_data[(elo_data.seasongame <= i) & (elo_data.fran_id == team) & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()
        - elo_data[(elo_data.fran_id == team) & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()).mean()))

In [ ]:
for i in range(1,82):
    diffs[i] = np.array(diffs[i]).mean()
    

In [ ]:
# These are the differences between season average ppg and average ppg through n games. 
%matplotlib inline
ppg_diffs = [abs(x[1]) for x in list(diffs.items())]
plt.plot(ppg_diffs)

In [ ]:
diffs = {}
for i in range(1,82):
    diffs[i] = []
for team in teams:
    for i in range(1,82):
        diffs[i].append(((elo_data[(elo_data.seasongame <= i) & (elo_data.fran_id == team) & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()
        - elo_data[(elo_data.fran_id == team) & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()).mean()))

In [ ]:
elo_data.columns

In [ ]:
elo_data[(elo_data.seasongame >= 42) & (elo_data.fran_id == 'Celtics') & (elo_data.is_playoffs == 0)].groupby('year_id')['pts'].mean()

In [ ]:
data_test

In [ ]:
data_t = data[data.Year <2016]

In [ ]:
model = RandomForestClassifier()
model.fit(data_train[inputs], data_train['Champion'])
model.predict_proba(data_test[inputs])

In [ ]:
data_fff = data[data.Year == 2016]
data_fff.drop('Champion',axis=1, inplace=True)
data_fff

In [ ]:
model = RandomForestClassifier()
model.fit(data_train[inputs], data_train['Champion'])

In [ ]:
model.predict_proba(data_fff[inputs])

In [ ]:
elo_data = pd.read_csv('./nbaallelo.csv')

In [ ]:
elo_data

In [ ]:
data[data.Year == 2016]['Team'].unique()

In [ ]:
model = RandomForestClassifier()

model.fit(data[data.Year < 2017][inputs], data[data.Year < 2017]['Champion'])

In [ ]:
max(model.predict_proba(data_test[inputs])[1])

In [ ]:
data_test.iloc[1]

In [ ]:
data['pred'] = model.predict(data[inputs])

In [ ]:
data

In [ ]:
data['pred'].sum()

In [ ]:
len(data['Year'].unique())

In [ ]:
data.iloc[1325]['All-Defensive']

In [ ]:
data[data['pred'] == 1]

In [ ]:
model.fit(data_train[data_train.Year != 2016][inputs], data_train[data_train.Year != 2016]['Champion'])

In [ ]:
for i in range(1960,2017):
    print(i,model.predict_proba(data[data.Year == i][inputs]))

In [6]:


inputs = ['Wins', 'MVP', 'DPOY', '6MOY', 'Coach of Year', 'Accolades', 'Scoring Leader',
          'All-NBA', 'RPG', 'TOVPG']
model = RandomForestClassifier(n_estimators = 10)
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(1960,2018):
    #print(i)
    teams = []
    model.fit(data_train[data_train.Year != i][inputs], data_train[data_train.Year != i]['Champion'])
    predictions = model.predict_proba(data[data.Year == i][inputs])
    predictions = [m[1] for m in predictions]    
    index, value = max(enumerate(predictions), key=operator.itemgetter(1))
    
    predicted = data[data.Year == i].iloc[index]['Team']
    #print('Predicted:',predicted)
    #print('Probability of Winning:', value*100 ,'%')
    actual = data[(data.Year == i) & (data.Champion == 1)].iloc[0]['Team']
    #print('Actual:',actual)
    if predicted == actual:
        TP += 1
        TN += len(predictions)-1
    if predicted != actual:
        FP += 1
        FN += 1
        TN += len(predictions)-2
    #print('---')
    
print('TP:',TP)
print('FP:',FP)
print('TN:',TN)
print('FN:',FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('F1:', 1/((1/precision + 1/recall)/2))



TP: 28
FP: 30
TN: 1239
FN: 30
F1: 0.48275862068965525


In [ ]:
inputs = ['Wins','MVP','Scoring Leader','Rebound Leader','Assist Leader',
          'WS Leader','MIP','6MOY','Coach of Year','All-Stars',
          'All-Defensive','All-NBA','FG%','3P Attempts','3P%',
          '2P%', 'FT%', 'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades'
         ]

scores = []
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
#Loop through all possible combinations of keywords. 
inpu = np.array(inputs)
for i in range(1,2**len(inpu)):
    mask = list(format(i, '016b'))
    mask = np.array([x=='1' for x in mask])
    combo = inpu[mask].tolist()
    
    for i in range(1960,2018):
        model.fit(data_train[data_train.Year != i][combo], data_train[data_train.Year != i]['Champion'])
        predictions = model.predict_proba(data[data.Year == i][combo])
        predictions = [m[1] for m in predictions]    
        index, value = max(enumerate(predictions), key=operator.itemgetter(1))

        predicted = data[data.Year == i].iloc[index]['Team']

        actual = data[(data.Year == i) & (data.Champion == 1)].iloc[0]['Team']
        if predicted == actual:
            TP += 1
            TN += len(predictions)-1
        if predicted != actual:
            FP += 1
            FN += 1
            TN += len(predictions)-2

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1 = ( 1/((1/precision + 1/recall)/2))

    scores.append((F1, combo))
    print(max(scores))
print(max(scores))


In [8]:
def choose_from_two(pred1, pred2, year):
    elo1 = elo_data[(elo_data.year_id == year) & (elo_data.fran_id == pred1)]
    elo2 = elo_data[(elo_data.year_id == year) & (elo_data.fran_id == pred2)]

    if elo1['elo_i'].mean() > elo2['elo_i'].mean():
        return pred1
    return pred2

In [9]:
# Need to beat 30 TP, which is what you would get if you only bet first seed

In [23]:
inputs = ['Wins','MVP','Scoring Leader','Rebound Leader','Assist Leader',
          'WS Leader','MIP','6MOY','Coach of Year','All-Stars',
          'All-Defensive','All-NBA','FG%','3P Attempts','3P%',
          '2P%', 'FT%', 'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG', 'Accolades'
         ]

inputs = ['Wins', 'MVP', 'DPOY', '6MOY', 'Coach of Year', 'Accolades', 'Scoring Leader',
          'All-NBA', 'RPG', 'TOVPG'
         ]

model = RandomForestClassifier(n_estimators = 50)
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(1960,2018):
    #print(i)
    teams = []
    model.fit(data_train[data_train.Year != i][inputs], data_train[data_train.Year != i]['Champion'])
    predictions = model.predict_proba(data[data.Year == i][inputs])
    predictions = [m[1] for m in predictions]    
    
    index, value = max(enumerate(predictions), key=operator.itemgetter(1))
    predicted1 = data[data.Year == i].iloc[index]['Franchise ID']
    predictions[index] = 0
    index2, value2 = max(enumerate(predictions), key=operator.itemgetter(1))
    predicted2 = data[data.Year == i].iloc[index2]['Franchise ID']
    
    predicted = choose_from_two(predicted1, predicted2, i)
    #print('Predicted:',predicted)
    
    actual = data[(data.Year == i) & (data.Champion == 1)].iloc[0]['Franchise ID']
    #print('Actual:',actual)
    if predicted1 == actual or predicted2 == actual:
        TP += 1
        TN += len(predictions)-1
    else:
        FP += 1
        FN += 1
        TN += len(predictions)-2
    #print('---')
    
print('TP:',TP)
print('FP:',FP)
print('TN:',TN)
print('FN:',FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('F1:', 1/((1/precision + 1/recall)/2))

TP: 38
FP: 20
TN: 1249
FN: 20
F1: 0.6551724137931034


In [ ]:
from sklearn.dummy import DummyClassifier
model = DummyClassifier()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(2001,2012):
    #print(i)
    teams = []
    model.fit(data_train[data_train.Year != i][inputs], data_train[data_train.Year != i]['Champion'])
    predictions = model.predict_proba(data[data.Year == i][inputs])
    predictions = [m[1] for m in predictions]    
    index, value = max(enumerate(predictions), key=operator.itemgetter(1))
    
    predicted = data[data.Year == i].iloc[index]['Team']
    #print('Predicted:',predicted)
    #print('Probability of Winning:', value*100 ,'%')
    actual = data[(data.Year == i) & (data.Champion == 1)].iloc[0]['Team']
    #print('Actual:',actual)
    if predicted == actual:
        TP += 1
        TN += len(predictions)-1
    if predicted != actual:
        FP += 1
        FN += 1
        TN += len(predictions)-2
    #print('---')
    
print('TP:',TP)
print('FP:',FP)
print('TN:',TN)
print('FN:',FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('F1:', 1/((1/precision + 1/recall)/2))



In [ ]:
model = RandomForestRegressor()

inputs = ['MVP','Scoring Leader','Rebound Leader','Assist Leader',
          'WS Leader','MIP','6MOY','Coach of Year','All-Stars',
          'All-Defensive','All-NBA','FG%','3P Attempts','3P%',
          '2P%', 'FT%', 'RPG', 'APG', 'STL', 'BLK', 'TOVPG', 'PPG'
         ]

print(-cross_val_score(model, data[inputs], data['Wins'], cv= 15,
                       scoring='neg_mean_squared_error').mean())

In [ ]:
model = RandomForestRegressor()

model.fit(data[inputs], data['Wins'])

In [ ]:
data['rf_pred'] = model.predict(data[inputs])

In [ ]:
data

In [ ]:
abs(data['rf_pred'] - data['Wins']).mean()

In [ ]:
abs(data['rf_pred'] - data['Wins']).std()